In [1]:
def getCandidateItems(transcation_weight): 
    candidateItems_count = {}
    candidateItems_weight = {}
    for items in transcation_weight:
        for item in items:
            key = frozenset((item[0],))
            if key not in candidateItems_weight.keys():
                candidateItems_count[key] = 1
                candidateItems_weight[key] = item[1]
            else:
                candidateItems_weight[key] = (candidateItems_weight[key]*candidateItems_count[key]+item[1])/(candidateItems_count[key]+1)
                candidateItems_count[key] = candidateItems_count[key] + 1
    return candidateItems_count,candidateItems_weight

In [2]:
def getFrequentItem(candidateItems_count,candidateItems_weight,total,frequency,weight,excellent_threshold,min_sup): 
    frequentItems_count = {}
    frequentItems_weight = {}
    for key in candidateItems_count:
        if ( (candidateItems_count[key]/total)*frequency + (candidateItems_weight[key] - excellent_threshold)*weight ) >= min_sup :
            frequentItems_count[key] = candidateItems_count[key]
            frequentItems_weight[key] = candidateItems_weight[key]
    return frequentItems_count,frequentItems_weight

In [3]:
def generateKCandidateItems(frequentItems_count,candidateItems_weight,candidateItems_count,transcation):
    kCandidateItems_count = {}
    kCandidateItems_weight = {}
    if(len(frequentItems_count) > 0):
        length = len(str(list(frequentItems_count.keys())[0]).split(',')) + 1
        for i in range(len(frequentItems_count)):
            for j in range(i+1,len(frequentItems_count)):
                newItem = list(frequentItems_count.keys())[i] | list(frequentItems_count.keys())[j]
                if(len(newItem) == length and newItem not in kCandidateItems_count.keys()):
                    found= False
                    for item in list(map(frozenset,transcation)):
                        if newItem.issubset(item):
                            if not found:
                                weight = 0.0
                                count = 0
                                for it in newItem:
                                    weight += candidateItems_weight[frozenset([it])] * candidateItems_count[frozenset([it])]
                                    count = count + candidateItems_count[frozenset([it])]
                                weight = weight/count
                                kCandidateItems_count[newItem] = 1
                                kCandidateItems_weight[newItem] = weight
                                found = True
                                #print("%s in %s：%s" % (newItem,item,weight))
                            else:
                                kCandidateItems_count[newItem] += 1
                                #print("%s：%s %s" % (newItem,kCandidateItems_count[newItem],kCandidateItems_weight[newItem]))
                        #else:
                            #print("%s not in %s" % (newItem,item))
                    #if(newItem in kCandidateItems_count.keys()):
                         #print("%s %s：%s %s" % (i,newItem,kCandidateItems_count[newItem],kCandidateItems_weight[newItem]))
                    #else:
                        #print("%s %s：0" % (i,newItem))
                #else:
                    #print("%s %s：重复" % (i,newItem,))
    return kCandidateItems_count,kCandidateItems_weight

In [4]:
def generateRules(kFrequentItems_weight,kFrequentItems_count,candidateItems_count,min_con):
    kRules = []
    ruleSet = []
    for kItems in kFrequentItems_count.keys():
        kRules = []
        for item in kItems:
            rear = frozenset({item})
            prev = kItems - rear
            #print(kItems)
            con = kFrequentItems_count[frozenset(kItems)] / candidateItems_count[frozenset(prev)]
            if(con >= min_con):
                kRules.append((list(prev),list(rear),con,kFrequentItems_weight[frozenset(kItems)]))
                #print((list(prev),list(rear),con))
        if(len(kRules) > 0):
            ruleSet.append(kRules)
        #生成后件项数为k的关联规则（k>1）
        #for k in range(2,len(kItems)):
            #kRules = generateNextRules(k,kItems,ruleSet[k-2],allSupCount,mincon)
            #if(len(kRules) > 0):
               #ruleSet.append(kRules)
    return ruleSet

In [5]:
def warm(transcation,transcation_weight,frequency,weight,excellent_threshold,min_sup,min_con):
    candidateItems_count,candidateItems_weight = getCandidateItems(transcation_weight = transcation_weight['Items'].values)
    #print(len(candidateItems_count),len(candidateItems_weight))#print(candidateItems_count)#print(candidateItems_weight)
    
    frequentItems_count,frequentItems_weight = getFrequentItem(candidateItems_count = candidateItems_count,
                                                               candidateItems_weight = candidateItems_weight,
                                                               total = float(len(transcation['Items'].values)),
                                                               min_sup = min_sup,
                                                               frequency = frequency,
                                                               excellent_threshold = excellent_threshold,
                                                               weight = weight)
    #print(len(frequentItems_count),len(frequentItems_count)) print(frequentItems_count) print(frequentItems_weight)
    
    kCandidateItems_count,kCandidateItems_weight = generateKCandidateItems(frequentItems_count = frequentItems_count,
                                                                           candidateItems_weight = candidateItems_weight,
                                                                           candidateItems_count = candidateItems_count,
                                                                           transcation=transcation['Items'].values)
    #print(len(kCandidateItems_count),len(kCandidateItems_weight)) print(kCandidateItems_count) print(kCandidateItems_weight)
    
    kFrequentItems_count,kFrequentItems_weight = getFrequentItem(candidateItems_count = kCandidateItems_count,
                                                                 candidateItems_weight = kCandidateItems_weight,
                                                                 total = float(len(transcation['Items'].values)),
                                                                 min_sup = min_sup,
                                                                 frequency = frequency,
                                                                 excellent_threshold = excellent_threshold,
                                                                 weight = weight)
    #print(len(kFrequentItems_count),len(kFrequentItems_weight)) print(kFrequentItems_count) print(kFrequentItems_weight)
    
    
    ruleSet = generateRules(kFrequentItems_weight = kFrequentItems_weight,
                            kFrequentItems_count = kFrequentItems_count,
                            candidateItems_count = candidateItems_count,
                            min_con = min_con)
    #print(len(ruleSet)) print(ruleSet)

    bases = []
    adds = []
    supports = []
    confidences = []
    lifts = []
    rule_warm = pd.DataFrame({
            'base': bases,
            'add': adds,
            'support': supports})
    for row in ruleSet:
        for rule in row:
            bases.append(rule[0])
            adds.append(rule[1])
            supports.append(rule[2])
    rule_warm['base'] = bases
    rule_warm['add'] = adds
    rule_warm['support'] = supports
    return rule_warm